In [2]:


import geopandas as gp
import pandas as pd
import os
import json
import pylab as pl
import statsmodels.api as sm

%pylab inline



Populating the interactive namespace from numpy and matplotlib


In [ ]:
nrg = gp.GeoDataFrame.from_csv("Energy_and_Water_Data_Disclosure_for_Local_Law_84__2013_.csv")
print nrg.columns

In [ ]:
bsize = gp.GeoDataFrame.from_file("Manhattan/MNMapPLUTO.shp")
print bsize.columns

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix (nrg, s=300, figsize=(16, 16), diagonal='kde')

In [ ]:
#create the indeces array here
#indx = ..

nrg['Reported Property Floor Area (Building(s)) (ft²)']=nrg['Reported Property Floor Area (Building(s)) (ft²)'].convert_objects(convert_numeric=True)
nrg['Site EUI(kBtu/ft2)']=nrg['Site EUI(kBtu/ft2)'].convert_objects(convert_numeric=True)

#I got convert_objects from discussion in class, multiple classmates provided advice


In [ ]:
#nrg['BBL'] = ...
#bblnrgdata = pd.merge(.....)

# merge datasets using newly created created column BBL
nrg['BBL'] = nrg['NYC Borough, Block, and Lot (BBL)']
bblnrgdata = nrg.merge(bsize, on='BBL')

In [ ]:
## Didn't like it when i used this

## bblnrgdata_cut = bblnrgdata[bblnrgdata.nrg>1000]

In [ ]:
## y = (bblnrgdata_cut['Site EUI(kBtu/ft2)'])*(bblnrgdata_cut['Reported Property Floor Area (Building(s)) (ft²)']) # total energy
## x = bblnrgdata_cut['UnitsTotal']

y = (bblnrgdata['Site EUI(kBtu/ft2)'])*(bblnrgdata['Reported Property Floor Area (Building(s)) (ft²)']) # total energy
x = bblnrgdata['UnitsTotal']

In [ ]:
fig=figure(figsize = (10,10))

ax = fig.add_subplot(111)
ax.plot(x, y, 'bo') #would I tack on loglog here?
ax.set_xlim(1,3000)
ax.set_ylim(1000, 1e11)
plt.xlabel('total units')
plt.ylabel('total energy/building')

In [ ]:
X = sm.add_constant(x)
print X
model = sm.OLS(y, X, missing='drop').fit() # ignores entires where x or y is NaN
fit = model.fit()
return fit.params[1], fit.params[0], fit # could also return stderr in each via fit.bse

In [ ]:
X = sm.add_constant(np.log10(x))
print X
model = sm.OLS(np.log10(y), X, missing='drop').fit() # ignores entires where x or y is NaN
fit = model.fit()
return fit.params[1], fit.params[0], fit # could also return stderr in each via fit.bse

In [ ]:

index = (x>0)&(x<4000)&(y>1000)&(y<1e11) # From Maria: must use ampersand, will assign true or false

In [ ]:
#plot model results above
Y = np.log10(y[index])

fig2 = figure(figsize = (10,10))
modX = np.arange(0, 4, 1)
modY = 0.0822*XLR + 6.9178
ax = fig2.add_subplot(111)
ax.plot(X, Y, 'ro')
ax.plot(modX, modY, 'blue')
plt.xlabel('log total units')
plt.ylabel('log total value')

1 var = 0 DF, therefore I can't do a likelihood ratio test. Log model fits better.